In [22]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE
import mplfinance as mpf
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from mplfinance.original_flavor import candlestick_ohlc
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import seaborn as sns
from finta import TA
import backtrader as bt
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy

In [10]:
# pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


In [23]:
## loading file

df = pd.read_csv(r"C:\Users\masho\OneDrive\Documents\Fall 24 SEM\Capstone\data\New folder\Latest data\BTC_training_data_2023-06-01_to_2023-11-30.csv")


In [24]:

# Calculating SMAs using TA-Lib
df['SMA_2'] = talib.SMA(df['close'], timeperiod=2)    # 2-period SMA
df['SMA_5'] = talib.SMA(df['close'], timeperiod=5)    # 5-period SMA
df['SMA_10'] = talib.SMA(df['close'], timeperiod=10)  # 10-period SMA
df['SMA_15'] = talib.SMA(df['close'], timeperiod=15)  # 15-period SMA
df['SMA_20'] = talib.SMA(df['close'], timeperiod=20)  # 20-period SMA
df['SMA_50'] = talib.SMA(df['close'], timeperiod=50)  # 50-period SMA

# Calculating SMAs using TA-Lib
df['EMA_2'] = talib.EMA(df['close'], timeperiod=2)    # 2-period EMA
df['EMA_5'] = talib.EMA(df['close'], timeperiod=5)    # 5-period EMA
df['EMA_10'] = talib.EMA(df['close'], timeperiod=10)  # 10-period EMA
df['EMA_15'] = talib.EMA(df['close'], timeperiod=15)  # 15-period EMA
df['EMA_20'] = talib.EMA(df['close'], timeperiod=20)  # 20-period EMA
df['EMA_50'] = talib.EMA(df['close'], timeperiod=50)  # 50-period EMA

# Displaying the DataFrame with the moving averages
df.head()


,unix,date,symbol,open,high,low,close,volume,volume_from,marketorder_volume,...,SMA_10,SMA_15,SMA_20,SMA_50,EMA_2,EMA_5,EMA_10,EMA_15,EMA_20,EMA_50
0,1.686610e+12,6/13/2023 0:00,BTCUSDT,25905.20,25909.22,25905.19,25905.37,11.71531,303516.0317,4.51426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.686610e+12,6/13/2023 0:01,BTCUSDT,25905.38,25905.38,25890.00,25890.00,24.30092,629364.2812,1.47995,...,NaN,NaN,NaN,NaN,25897.685000,NaN,NaN,NaN,NaN,NaN
2,1.686610e+12,6/13/2023 0:02,BTCUSDT,25890.01,25895.81,25890.00,25892.96,24.58701,636642.5209,11.10671,...,NaN,NaN,NaN,NaN,25894.535000,NaN,NaN,NaN,NaN,NaN
3,1.686610e+12,6/13/2023 0:03,BTCUSDT,25892.96,25907.34,25892.95,25907.34,10.07185,260841.6340,9.70316,...,NaN,NaN,NaN,NaN,25903.071667,NaN,NaN,NaN,NaN,NaN
4,1.686610e+12,6/13/2023 0:04,BTCUSDT,25907.33,25914.21,25907.33,25914.21,9.34626,242160.6105,8.56727,...,NaN,NaN,NaN,NaN,25910.497222,25901.976,NaN,NaN,NaN,NaN


In [5]:
df.tail()

,unix,date,symbol,open,high,low,close,volume,volume_from,marketorder_volume,...,SMA_10,SMA_15,SMA_20,SMA_50,EMA_2,EMA_5,EMA_10,EMA_15,EMA_20,EMA_50
79195,1.700440e+12,11/19/2023 23:55,BTCUSDT,37388.63,37399.99,37388.63,37399.97,14.84403,5.551363e+05,9.35623,...,37420.742,37439.684667,37441.7625,37419.3732,37395.484161,37398.873456,37413.142953,37420.812174,37422.524142,37369.462367
79196,1.700440e+12,11/19/2023 23:56,BTCUSDT,37399.97,37414.98,37394.13,37412.17,20.06398,7.504741e+05,10.87734,...,37414.300,37435.304667,37439.3780,37422.0964,37406.608054,37403.305637,37412.966053,37419.731902,37421.538033,37371.137177
79197,1.700440e+12,11/19/2023 23:57,BTCUSDT,37412.17,37412.18,37364.00,37364.00,30.23194,1.130147e+06,5.15366,...,37404.222,37426.323333,37435.1845,37422.4608,37378.202685,37390.203758,37404.063134,37412.765414,37416.058220,37370.857287
79198,1.700440e+12,11/19/2023 23:58,BTCUSDT,37364.01,37369.95,37345.26,37366.59,23.95295,8.949373e+05,12.01290,...,37394.881,37418.474000,37432.2985,37423.0198,37370.460895,37382.332505,37397.249837,37406.993487,37411.346961,37370.689943
79199,1.700440e+12,11/19/2023 23:59,BTCUSDT,37366.58,37366.59,37349.18,37359.86,31.46003,1.175153e+06,15.20176,...,37388.959,37410.785333,37428.6360,37423.0670,37363.393632,37374.841670,37390.451685,37401.101802,37406.443441,37370.265239


In [6]:
df['date'] = pd.to_datetime(df['date'])

# Set 'datetime' as the index
df.set_index('date', inplace=True)
df.head()

,unix,symbol,open,high,low,close,volume,volume_from,marketorder_volume,marketorder_volume_from,...,SMA_10,SMA_15,SMA_20,SMA_50,EMA_2,EMA_5,EMA_10,EMA_15,EMA_20,EMA_50
date,,,,,,,,,,,,,,,,,,,,,
2023-06-13 00:00:00,1.686610e+12,BTCUSDT,25905.20,25909.22,25905.19,25905.37,11.71531,303516.0317,4.51426,116949.07450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-13 00:01:00,1.686610e+12,BTCUSDT,25905.38,25905.38,25890.00,25890.00,24.30092,629364.2812,1.47995,38322.32746,...,NaN,NaN,NaN,NaN,25897.685000,NaN,NaN,NaN,NaN,NaN
2023-06-13 00:02:00,1.686610e+12,BTCUSDT,25890.01,25895.81,25890.00,25892.96,24.58701,636642.5209,11.10671,287567.08850,...,NaN,NaN,NaN,NaN,25894.535000,NaN,NaN,NaN,NaN,NaN
2023-06-13 00:03:00,1.686610e+12,BTCUSDT,25892.96,25907.34,25892.95,25907.34,10.07185,260841.6340,9.70316,251292.13920,...,NaN,NaN,NaN,NaN,25903.071667,NaN,NaN,NaN,NaN,NaN
2023-06-13 00:04:00,1.686610e+12,BTCUSDT,25907.33,25914.21,25907.33,25914.21,9.34626,242160.6105,8.56727,221974.90230,...,NaN,NaN,NaN,NaN,25910.497222,25901.976,NaN,NaN,NaN,NaN


In [7]:
df.tail()

,unix,symbol,open,high,low,close,volume,volume_from,marketorder_volume,marketorder_volume_from,...,SMA_10,SMA_15,SMA_20,SMA_50,EMA_2,EMA_5,EMA_10,EMA_15,EMA_20,EMA_50
date,,,,,,,,,,,,,,,,,,,,,
2023-11-19 23:55:00,1.700440e+12,BTCUSDT,37388.63,37399.99,37388.63,37399.97,14.84403,5.551363e+05,9.35623,349898.8905,...,37420.742,37439.684667,37441.7625,37419.3732,37395.484161,37398.873456,37413.142953,37420.812174,37422.524142,37369.462367
2023-11-19 23:56:00,1.700440e+12,BTCUSDT,37399.97,37414.98,37394.13,37412.17,20.06398,7.504741e+05,10.87734,406846.5342,...,37414.300,37435.304667,37439.3780,37422.0964,37406.608054,37403.305637,37412.966053,37419.731902,37421.538033,37371.137177
2023-11-19 23:57:00,1.700440e+12,BTCUSDT,37412.17,37412.18,37364.00,37364.00,30.23194,1.130147e+06,5.15366,192650.9890,...,37404.222,37426.323333,37435.1845,37422.4608,37378.202685,37390.203758,37404.063134,37412.765414,37416.058220,37370.857287
2023-11-19 23:58:00,1.700440e+12,BTCUSDT,37364.01,37369.95,37345.26,37366.59,23.95295,8.949373e+05,12.01290,448803.5885,...,37394.881,37418.474000,37432.2985,37423.0198,37370.460895,37382.332505,37397.249837,37406.993487,37411.346961,37370.689943
2023-11-19 23:59:00,1.700440e+12,BTCUSDT,37366.58,37366.59,37349.18,37359.86,31.46003,1.175153e+06,15.20176,567813.2613,...,37388.959,37410.785333,37428.6360,37423.0670,37363.393632,37374.841670,37390.451685,37401.101802,37406.443441,37370.265239


In [8]:
# Creating RSI for different time periods

# Calculating RSI for scalping (7-period)
df['RSI_7'] = talib.RSI(df['close'], timeperiod=7)

# Calculating RSI for day trading (14-period)
df['RSI_14'] = talib.RSI(df['close'], timeperiod=14)

# Calculating RSI for swing trading (21-period)
df['RSI_21'] = talib.RSI(df['close'], timeperiod=21)

# Displaying the DataFrame with the new RSI columns
df[['close', 'RSI_7', 'RSI_14', 'RSI_21']].head(n=10)


,close,RSI_7,RSI_14,RSI_21
date,,,,
2023-06-13 00:00:00,25905.37,NaN,NaN,NaN
2023-06-13 00:01:00,25890.00,NaN,NaN,NaN
2023-06-13 00:02:00,25892.96,NaN,NaN,NaN
2023-06-13 00:03:00,25907.34,NaN,NaN,NaN
2023-06-13 00:04:00,25914.21,NaN,NaN,NaN
2023-06-13 00:05:00,25904.49,NaN,NaN,NaN
2023-06-13 00:06:00,25908.97,NaN,NaN,NaN
2023-06-13 00:07:00,25904.50,49.253219,NaN,NaN
2023-06-13 00:08:00,25895.24,41.547596,NaN,NaN


In [9]:
# Calculating MACD with adjusted periods (e.g., 7, 21, 5)
df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(df['close'], fastperiod=7, slowperiod=21, signalperiod=5)

# Displaying the DataFrame with MACD, Signal Line, and Histogram columns
df[['close', 'MACD', 'MACD_Signal', 'MACD_Hist']].tail()


,close,MACD,MACD_Signal,MACD_Hist
date,,,,
2023-11-19 23:55:00,37399.97,-17.173802,-10.577743,-6.596058
2023-11-19 23:56:00,37412.17,-14.487143,-11.880876,-2.606266
2023-11-19 23:57:00,37364.00,-19.989486,-14.583746,-5.405740
2023-11-19 23:58:00,37366.59,-22.874732,-17.347408,-5.527324
2023-11-19 23:59:00,37359.86,-25.392746,-20.029187,-5.363559


In [10]:
# Bollinger Bands with 10 periods and 1.5 standard deviations
df['upper_band_10_1_5'], df['middle_band_10_1_5'], df['lower_band_10_1_5'] = talib.BBANDS(
    df['close'], timeperiod=10, nbdevup=1.5, nbdevdn=1.5, matype=1)

# Bollinger Bands with 10 periods and 2 standard deviations
df['upper_band_10_2'], df['middle_band_10_2'], df['lower_band_10_2'] = talib.BBANDS(
    df['close'], timeperiod=10, nbdevup=2, nbdevdn=2, matype=1)

# Bollinger Bands with 20 periods and 1.5 standard deviations
df['upper_band_20_1_5'], df['middle_band_20_1_5'], df['lower_band_20_1_5'] = talib.BBANDS(
    df['close'], timeperiod=20, nbdevup=1.5, nbdevdn=1.5, matype=1)

# Bollinger Bands with 20 periods and 2 standard deviations
df['upper_band_20_2'], df['middle_band_20_2'], df['lower_band_20_2'] = talib.BBANDS(
    df['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=1)


In [34]:
# pip install xlsxwriter


Note: you may need to restart the kernel to use updated packages.


In [14]:
# Renaming columns to fit the Backtesting.py requirements
df = df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'  
})

In [20]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Strategy classes remain unchanged (as defined in your code)
# 1. SMA(2,5)
class SMA25Strategy(Strategy):
    def init(self):
        self.sma2 = self.I(lambda x: x, self.data.SMA_2)
        self.sma5 = self.I(lambda x: x, self.data.SMA_5)

    def next(self):
        if crossover(self.sma2, self.sma5):
            if not self.position:
                self.buy()
        elif crossover(self.sma5, self.sma2):
            if self.position:
                self.position.close()

# 2. SMA(5,10)
class SMA510Strategy(Strategy):
    def init(self):
        self.sma5 = self.I(lambda x: x, self.data.SMA_5)
        self.sma10 = self.I(lambda x: x, self.data.SMA_10)

    def next(self):
        if crossover(self.sma5, self.sma10):
            if not self.position:
                self.buy()
        elif crossover(self.sma10, self.sma5):
            if self.position:
                self.position.close()

# 3. RSI(40-60)
class RSI40_60_Strategy(Strategy):
    def init(self):
        self.rsi = self.I(lambda x: x, self.data.RSI_14)

    def next(self):
        if self.rsi[-1] < 40:
            if not self.position:
                self.buy()
        elif self.rsi[-1] > 60:
            if self.position:
                self.position.close()

# 4. MACD (7,21,5)
class MACDStrategy(Strategy):
    def init(self):
        self.macd = self.I(lambda x: x, self.data.MACD)
        self.macd_signal = self.I(lambda x: x, self.data.MACD_Signal)

    def next(self):
        if crossover(self.macd, self.macd_signal):
            if not self.position:
                self.buy()
        elif crossover(self.macd_signal, self.macd):
            if self.position:
                self.position.close()

# 5. Bollinger Bands (20,2)
class BollingerBandsStrategy(Strategy):
    def init(self):
        self.upper_band = self.I(lambda x: x, self.data.upper_band_20_2)
        self.lower_band = self.I(lambda x: x, self.data.lower_band_20_2)

    def next(self):
        if self.data.Close[-1] < self.lower_band[-1]:
            if not self.position:
                self.buy()
        elif self.data.Close[-1] > self.upper_band[-1]:
            if self.position:
                self.position.close()

# 6. SMA-EMA(2,5)
class SMAEMA25Strategy(Strategy):
    def init(self):
        self.sma2 = self.I(lambda x: x, self.data.SMA_2)
        self.ema5 = self.I(lambda x: x, self.data.EMA_5)

    def next(self):
        if crossover(self.sma2, self.ema5):
            if not self.position:
                self.buy()
        elif crossover(self.ema5, self.sma2):
            if self.position:
                self.position.close()

# 7. RSI-MACD
class RSIMACDStrategy(Strategy):
    def init(self):
        self.rsi = self.I(lambda x: x, self.data.RSI_14)
        self.macd = self.I(lambda x: x, self.data.MACD)
        self.macd_signal = self.I(lambda x: x, self.data.MACD_Signal)

    def next(self):
        if self.rsi[-1] < 40 and crossover(self.macd, self.macd_signal):
            if not self.position:
                self.buy()
        elif self.rsi[-1] > 60 or crossover(self.macd_signal, self.macd):
            if self.position:
                self.position.close()

# 8. EMA-BB
class EMABBStrategy(Strategy):
    def init(self):
        self.ema20 = self.I(lambda x: x, self.data.EMA_20)
        self.upper_band = self.I(lambda x: x, self.data.upper_band_20_2)
        self.lower_band = self.I(lambda x: x, self.data.lower_band_20_2)

    def next(self):
        if self.ema20[-1] < self.lower_band[-1]:
            if not self.position:
                self.buy()
        elif self.ema20[-1] > self.upper_band[-1]:
            if self.position:
                self.position.close()

# Defining strategies dictionary
strategies = {
    'SMA_2_5': SMA25Strategy,
    'SMA_5_10': SMA510Strategy,
    'RSI_40_60': RSI40_60_Strategy,
    'MACD': MACDStrategy,
    'BB_20_2': BollingerBandsStrategy,
    'SMA_EMA_2_5': SMAEMA25Strategy,
    'RSI_MACD': RSIMACDStrategy,
    'EMA_BB': EMABBStrategy
}

# Fetching data for a specific time frame
def fetch_data_for_time_frame(df, start_date, time_frame):
    if 'min' in time_frame:
        minutes = int(time_frame.replace('min', ''))
        end_date = start_date + pd.Timedelta(minutes=minutes)
    elif 'hr' in time_frame:
        hours = int(time_frame.replace('hr', ''))
        end_date = start_date + pd.Timedelta(hours=hours)
    elif 'week' in time_frame:
        weeks = int(time_frame.replace('week', ''))
        end_date = start_date + pd.Timedelta(weeks=weeks)
    elif 'day' in time_frame or '24hr' in time_frame:
        end_date = start_date + pd.Timedelta(days=1)
    else:
        return None, None
    mask = (df.index >= start_date) & (df.index < end_date)
    return df.loc[mask], end_date


# Generalizing function for running and saving backtest
def run_and_save_backtest(strategy_class, df_backtest):
    bt = Backtest(df_backtest, strategy_class, cash=10000, commission=.002, exclusive_orders=True)
    output = bt.run()
    return output

# Functioning to calculate Sharpe Ratio, Annualized Volatility, Annualized Return, Market Return, and Max Drawdown
def calculate_metrics(results, periods_per_year):
    metrics = []  # To store results for each timeframe

    for start, end, output in results:
        # Extracting the equity curve
        equity_curve = output['_equity_curve']

        # Calculating returns
        equity_curve['Return'] = equity_curve['Equity'].pct_change()  # Percent change in equity
        returns = equity_curve['Return'].dropna()  # Drop NaN values

        # Calculating cumulative return
        starting_equity = equity_curve['Equity'].iloc[0]
        ending_equity = equity_curve['Equity'].iloc[-1]
        cumulative_return = (ending_equity - starting_equity) / starting_equity

        # Calculating total periods
        total_periods = len(equity_curve)

        # Calculating metrics
        mean_return = returns.mean()
        std_dev = returns.std()
        sharpe_ratio = mean_return / std_dev if std_dev != 0 else 0  # Avoid division by zero
        annualized_sharpe = sharpe_ratio * (periods_per_year ** 0.5)
        annualized_volatility = std_dev * (periods_per_year ** 0.5)
        annualized_return = (1 + cumulative_return) ** (periods_per_year / total_periods) - 1

        # Market return calculation (benchmark return)
        market_return = cumulative_return  # Assuming 'Equity' tracks the market index

        # Max drawdown calculation
        cumulative_max = equity_curve['Equity'].cummax()  # Cumulative maximum equity
        drawdown = (equity_curve['Equity'] - cumulative_max) / cumulative_max  # Drawdown calculation
        max_drawdown = drawdown.min()  # Maximum drawdown (most negative value)

        # Saving results
        metrics.append({
            'Start': start,
            'End': end,
            'Mean Return': mean_return,
            'Standard Deviation': std_dev,
            'Sharpe Ratio': sharpe_ratio,
            'Annualized Sharpe Ratio': annualized_sharpe,
            'Annualized Volatility': annualized_volatility,
            'Annualized Return': annualized_return,
            'Market Return': market_return,
            'Max Drawdown': max_drawdown,
            'Equity Final [$]': output['Equity Final [$]'],
            '# Trades': output['# Trades']
        })

    return pd.DataFrame(metrics)


# Defining periods per year based on the timeframe
time_frame_mapping = {
    '15min': 525600 / 15,
    '1hr': 8760,
    '4hr': 8760 / 4,
    '8hr': 8760 / 8,
    '24hr': 365,
    '1 week': 52
}


# Main execution
if __name__ == "__main__":
    time_frame = input("Enter the desired time frame (e.g., '15min', '1hr', '4hr', '8hr', '24hr', '1 week'): ").strip()

    if time_frame not in time_frame_mapping:
        print(f"Unsupported timeframe: {time_frame}")
        exit()

    periods_per_year = time_frame_mapping[time_frame]

    results = []
    start_date = df.index.min()
    while start_date < df.index.max():
        filtered_data, end_date = fetch_data_for_time_frame(df, start_date, time_frame)
        if filtered_data is None or filtered_data.empty:
            break
        df_backtest = filtered_data.loc[filtered_data['EMA_5'].first_valid_index():].copy()

        if df_backtest.empty:
            start_date = end_date
            continue

        max_close = df_backtest['Close'].max()
        target_max_price = 100
        scale_factor = max_close / target_max_price

        df_backtest['Open'] /= scale_factor
        df_backtest['High'] /= scale_factor
        df_backtest['Low'] /= scale_factor
        df_backtest['Close'] /= scale_factor

        for name, strategy in strategies.items():
            output = run_and_save_backtest(strategy, df_backtest)
            results.append((start_date, end_date, output, name))

        start_date = end_date

    # Processing and saving results for each strategy
    for strategy_name in strategies.keys():
        # Extract results for the strategy
        strategy_results = [(start, end, output) for start, end, output, name in results if name == strategy_name]

        # Filter results with more than 1 trade
        results_with_trades = [(start, end, output) for start, end, output in strategy_results if output['# Trades'] > 1]

        # Sort by Equity Final [$] in descending order
        sorted_results = sorted(results_with_trades, key=lambda x: x[2]['Equity Final [$]'], reverse=True)[:3]  # Top 3

        # Calculate metrics
        metrics = calculate_metrics(sorted_results, periods_per_year)

        # Save to CSV
        metrics.to_csv(f'{strategy_name}_metrics.csv', index=False)

        print(f"Saved top 3 trades with metrics for {strategy_name} to {strategy_name}_metrics.csv")


Enter the desired time frame (e.g., '15min', '1hr', '4hr', '8hr', '24hr', '1 week'):  1 week


Saved top 3 trades with metrics for SMA_2_5 to SMA_2_5_metrics.csv
Saved top 3 trades with metrics for SMA_5_10 to SMA_5_10_metrics.csv
Saved top 3 trades with metrics for RSI_40_60 to RSI_40_60_metrics.csv
Saved top 3 trades with metrics for MACD to MACD_metrics.csv
Saved top 3 trades with metrics for BB_20_2 to BB_20_2_metrics.csv
Saved top 3 trades with metrics for SMA_EMA_2_5 to SMA_EMA_2_5_metrics.csv
Saved top 3 trades with metrics for RSI_MACD to RSI_MACD_metrics.csv
Saved top 3 trades with metrics for EMA_BB to EMA_BB_metrics.csv


In [ ]:
###########)))))))))))))))))##########################

In [ ]:
##########3MMMMMMMMMMMMMMM#

In [ ]:
########################################

In [1]:
######################################3